# Migrer variabeldefinisjoner

Denne Notebook er for deg som:

- Har en eller flere variabeldefinisjoner i Vardok og ønsker å migrere de til Vardef

Forutsetninger:

- Du må ha alle `id`'er i Vardok for variabeldefinisjonene du ønsker å migrere

## Oppsett

Koden under kjøres som forberedelse for påfølgende steg

In [ ]:
# Nødvendig import
import logging
import sys

from dapla_metadata.variable_definitions import Vardef, VardefClientError

# Redusere størrelsen på Traceback for mer tydelige feilmeldinger
%xmode Minimal

# Gjøre at logging vises
logging.basicConfig(
    format="%(levelname)s: %(message)s",
    level=logging.INFO,
    stream=sys.stdout,
    force=True,
)

## Viktig informasjon

Når en variabeldefinisjon fra Vardok migreres til Vardef, blir den alltid opprettet som et `UTKAST`, uavhengig av hvilken status den hadde i Vardok. Dette gjøres for at man skal kunne oppdatere og tilpasse variabeldefinisjonen slik at den oppfyller kravene for publisering i Vardef.

⚠️ **En variabeldefinisjon kan kun migreres én gang.**
Når en Vardok-ID er migrert til Vardef, vil det ikke være mulig å migrere den samme ID-en på nytt.

📅 **Merk:** Dersom variabeldefinisjonen i Vardok mangler en gyldig fra-dato, settes denne automatisk til `1900-01-01`.

🏷️ **Kortnavn:** Hvis det ikke finnes et kortnavn for variabeldefinisjonen i Vardok, vil det bli automatisk generert under migreringen. 

## Hente variabeldefinisjoner fra Vardok 

Skriv inn de Vardok-ID'ene du ønsker å migrere i firkantparentesen. Erstatt 0 med en eller flere eksisterende Vardok-ID'er adskilt av komma

In [ ]:
min_id_liste = [0]
print(f"✅ {len(min_id_liste)} vil bli migrert")

## Migrere variabeldefinisjoner til Vardef

⭐️ I koden under migreres variablene fra Vardok til Vardef for Vardok-ID'ene du skrev inn

In [ ]:
migrerte = []
ikke_funnet = []
migrert_tidligere = []
annen_feil = []
for min_id in min_id_liste:
    try:
        mitt_kortnavn = Vardef.migrate_from_vardok(str(min_id))
        migrerte.append(mitt_kortnavn)
    except VardefClientError as e:
        if "not found" in str(e):
            ikke_funnet.append(min_id)
        elif "already migrated" in str(e):
            migrert_tidligere.append(min_id)
        else:
            print(f"⚠️ Ukjent VardefClientError for id {min_id}: {e}")
            annen_feil.append((min_id, str(e)))
    except Exception as e:
        print(f"❌ Uventet feil for id {min_id}: {e}")
        annen_feil.append((min_id, str(e)))

print(f"\n📊 Resultatoppsummering:")
print(f" - Antall migrert: {len(migrerte)}")
print(f" - Antall ikke funnet: {len(ikke_funnet)}")
print(f" - Antall allerede migrert: {len(migrert_tidligere)}")
print(f" - Antall med annen feil: {len(annen_feil)}\n")

### Detaljert oversikt

Koden under gir en detaljert oversikt over hva som skjedde med hver Vardok-ID under migrering der migreringen ikke gikk bra

In [ ]:
print("\nIkke fullført migrering:")

print("\n❌ Variabeldefinisjon eksisterer ikke:")
if ikke_funnet:
    for p in ikke_funnet:
        print(f" - {p}")
else:
    print(" - Ingen")

print("\n⚠️ Allerede migrert:")
if migrert_tidligere:
    for p in migrert_tidligere:
        print(f" - {p}")
else:
    print(" - Ingen")

print("\n🚨 Øvrige feil:")
if annen_feil:
    for p, feilmelding in annen_feil:
        print(f" - id {p}: {feilmelding}")
else:
    print(" - Ingen")